In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
files = os.listdir("Modules")
files = [file for file in files if ".v" in file]

## If file already exist then overwrite or not

In [15]:
file = open("Script/tcl_add.tcl","w")
current_directory = os.getcwd()
path = os.path.join(current_directory, 'Assignment')
if(os.path.exists(path)):
    print("Assignment file already exists do you want to overwrite it (yes/no) :")
    x=input()
    if(x=="yes"):
        file.write("create_project -force Assignment [pwd]/Assignment -part xc7a35tcpg236-1 \n")
    else:
        file.write("open_project [pwd]/Assignment/Assignment.xpr \n")
else:
    file.write("create_project -force Assignment [pwd]/Assignment -part xc7a35tcpg236-1 \n")

file.write("set_property board_part digilentinc.com:basys3:part0:1.1 [current_project] \n")
file.write("update_compile_order -fileset sources_1 \n")
file.write("foreach m_file $argv { \n")
file.write("    add_files -norecurse $m_file \n")
file.write("        update_compile_order -fileset sources_1 \n")
file.write("} \n")
file.write("close_project \n")
file.close()

Assignment file already exists do you want to overwrite it (yes/no) :


 no


## Add Files

In [3]:
v_file = ""
for module in files:
    v_file += "Modules/" + module + " "

In [4]:
os.system("vivado -mode batch -source Script/tcl_add.tcl -tclargs {}".format(v_file))

0

In [5]:
os.mkdir("Impl")

## Find Top Modules

In [6]:
top_modules = [file for file in files if "top" in file]
top_modules

['top_Array_Multiplier_4bit.v',
 'top_CLA8.v',
 'top_CLA16.v',
 'top_CLA32.v',
 'top_mealy.v',
 'top_moore.v',
 'top_Priority_Encoder8_3.v']

## Create TOP Module Directory in Impl

In [7]:
for mod in top_modules:
    os.mkdir("Impl/" + mod[:-2])

In [8]:
for mod in top_modules:
    os.system("vivado -mode batch -source Script/tcl_run.tcl -tclargs {}".format(mod))
    print("{} is Implmented and Reports are generated".format(mod))

top_Array_Multiplier_4bit.v is Implmented and Reports are generated
top_CLA8.v is Implmented and Reports are generated
top_CLA16.v is Implmented and Reports are generated
top_CLA32.v is Implmented and Reports are generated
top_mealy.v is Implmented and Reports are generated
top_moore.v is Implmented and Reports are generated
top_Priority_Encoder8_3.v is Implmented and Reports are generated


In [9]:
DatapathDelay = []

DynamicPower = []
SignalPower = []
LogicPower = []
StaticPower = []
TotalPower = []
DCPower = []

LUT = []
Slice_LUTs = []
Slice_LUTs2 = []
F7_muxes = []
LUT_as_Memory = []
LUT_as_Memory2 = []
LUT_as_Logic = []
LUT_as_Logic2 = []
LUTs_difference = []
n = 0
for mod in top_modules:
    n = n + 1
    with open("Impl/{}/timing.txt".format(mod[:-2])) as f:
        lines = f.read()
        timing = lines.split("\n")
        timing_info = [line for line in timing if "Data Path Delay" in line]
        dpd = float(timing_info[0].split()[3][:-2])
        DatapathDelay.append(dpd)
        
    with open("Impl/{}/power.txt".format(mod[:-2])) as f:
        lines = f.read()
        power = lines.split("\n")
        power_info_1 = [line for line in power if "Dynamic" in line]
        dpower = float(power_info_1[0].split("|")[2].strip())
        DynamicPower.append(dpower)
        
        power_info_2 = [line for line in power if "Signals" in line]
        sigpower = float(power_info_2[0].split("|")[2].strip())
        SignalPower.append(sigpower)
        
        power_info_3 = [line for line in power if "Slice Logic" in line]
        logpower = float(power_info_3[0].split("|")[2].strip())
        LogicPower.append(logpower)
        
        power_info_4 = [line for line in power if "Static Power" in line]
        statpower = float(power_info_4[0].split("|")[2].strip())
        StaticPower.append(statpower)
        
        power_info_5 = [line for line in power if "Total" in line]
        totpower = float(power_info_5[1].split("|")[2].strip())
        TotalPower.append(totpower)
        
        dcpower = (totpower-statpower)
        DCPower.append(dcpower)
        
    with open("Impl/{}/utilization.txt".format(mod[:-2])) as f:
        lines = f.read()
        uti = lines.split("\n")
        
        uti_info_1 = [line for line in uti if "Slice LUTs" in line]
        util_1 = int(uti_info_1[0].split("|")[2].strip())
        Slice_LUTs.append(util_1)
        
        uti_info_2 = [line for line in uti if "F7 Muxes" in line]
        util_2 = int(uti_info_2[0].split("|")[2].strip())
        F7_muxes.append(util_2)
        
        uti_info_3 = [line for line in uti if "LUT as Memory" in line]
        util_3 = int(uti_info_3[0].split("|")[2].strip())
        LUT_as_Memory.append(util_3)

        uti_info_4 = [line for line in uti if "LUT as Logic" in line]
        util_4 = int(uti_info_4[0].split("|")[2].strip())
        LUT_as_Logic.append(util_4)
        
        uti_info_5 = [line for line in uti if "Slice" in line]
        util_5 = int(uti_info_5[6].split("|")[2].strip())
        Slice_LUTs2.append(util_5)
        
        uti_info_6 = [line for line in uti if "LUT as Logic" in line]
        util_6 = int(uti_info_6[1].split("|")[2].strip())
        LUT_as_Logic2.append(util_6)
        
        unit_7 = int(util_6 - util_4)
        LUTs_difference.append(unit_7)
        
        uti_info_8 = [line for line in uti if "LUT as Memory" in line]
        util_8 = int(uti_info_8[0].split("|")[2].strip())
        LUT_as_Memory2.append(util_8)

        for x in range(1,8):
            lut_info = [line for line in uti if "LUT{}".format(x) in line]
            if len(lut_info)==0:
                LUT.append(0)
            else:
                luts = int(lut_info[0].split("|")[2].strip())
                LUT.append(luts)
                
LUT1 = [LUT[0+7*i] for i in range(n)]
LUT2 = [LUT[1+7*i] for i in range(n)]
LUT3 = [LUT[2+7*i] for i in range(n)]
LUT4 = [LUT[3+7*i] for i in range(n)]
LUT5 = [LUT[4+7*i] for i in range(n)]
LUT6 = [LUT[5+7*i] for i in range(n)]
LUT7 = [LUT[6+7*i] for i in range(n)]

In [10]:
result = [DynamicPower, SignalPower, LogicPower, StaticPower, DCPower, TotalPower, DatapathDelay, F7_muxes, LUT_as_Memory, LUT_as_Logic, Slice_LUTs, LUT1, LUT2, LUT3, LUT4, LUT5, LUT6, LUT7, LUT_as_Memory2, Slice_LUTs2, LUT_as_Logic2, LUTs_difference]          
resultT = np.array(result).T
heading = ["Dynamic Power (W)", "Signal Power (W)", "Logic Power (W)", "Static Power (W)", "DC Power (W)", "Total Power (W)", "Data Path Delay (ns)", "F7 Muxes", "LUT as Memory", "LUT as Logic", "Slice LUTs", "LUT1", "LUT2", "LUT3", "LUT4", "LUT5", "LUT6", "LUT7", "LUT as Memory2", "Slice LUTs2", "LUT as Logic2", "LUTs Difference"]

In [11]:
df = pd.DataFrame(resultT)

In [12]:
df.columns = heading

In [13]:
df.insert(loc=0, column='Design', value=top_modules)

In [14]:
df

,Design,Dynamic Power (W),Signal Power (W),Logic Power (W),Static Power (W),DC Power (W),Total Power (W),Data Path Delay (ns),F7 Muxes,LUT as Memory,...,LUT2,LUT3,LUT4,LUT5,LUT6,LUT7,LUT as Memory2,Slice LUTs2,LUT as Logic2,LUTs Difference
0,top_Array_Multiplier_4bit.v,3.951,0.133,0.085,0.083,3.951,4.034,10.902,0.0,0.0,...,7.0,1.0,1.0,5.0,9.0,0.0,0.0,5.0,17.0,0.0
1,top_CLA8.v,4.807,0.167,0.065,0.088,4.807,4.895,9.882,0.0,0.0,...,1.0,2.0,3.0,3.0,6.0,0.0,0.0,4.0,13.0,0.0
2,top_CLA16.v,10.890,0.500,0.194,0.156,10.890,11.046,12.463,0.0,0.0,...,6.0,4.0,8.0,5.0,22.0,0.0,0.0,10.0,35.0,0.0
3,top_CLA32.v,22.729,1.749,0.505,0.485,22.728,23.213,18.806,0.0,0.0,...,26.0,9.0,14.0,15.0,59.0,0.0,0.0,33.0,96.0,0.0
4,top_mealy.v,0.222,0.140,0.078,0.071,0.221,0.292,6.305,0.0,0.0,...,66.0,11.0,3.0,1.0,0.0,0.0,0.0,35.0,66.0,0.0
5,top_moore.v,0.222,0.140,0.078,0.069,0.222,0.291,6.177,0.0,0.0,...,66.0,11.0,1.0,4.0,0.0,0.0,0.0,35.0,67.0,0.0
6,top_Priority_Encoder8_3.v,0.386,0.041,0.013,0.071,0.386,0.457,7.676,0.0,0.0,...,1.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,4.0,0.0


In [16]:
df.to_csv("result.csv", index = False)